In [1]:
body = [{"x": 0, "y": 0}, 
        {"x": 1, "y": 0}, 
        {"x": 2, "y": 0}]

In [2]:
body2 = [{"x": 0, "y": 0}, 
        {"x": 1, "y": 0}, 
        {"x": 2, "y": 0}]

In [3]:
body3 = [{"x": 0, "y": 0}, 
        {"x": 1, "y": 0}, 
        {"x": 2, "y": 3}]

In [4]:
opponents = [body,body2,body3]
op_body = []

In [5]:
for x in opponents:
    op_body += x[:-1]

In [6]:
body + [{'x': 9, 'y':9}]

[{'x': 0, 'y': 0}, {'x': 1, 'y': 0}, {'x': 2, 'y': 0}, {'x': 9, 'y': 9}]

In [ ]:
'''
L L L L L L L L L L L
L L L L L L L L L L L
L L L L L L L L L L L
L L L L L L L L L L L
L L L L L L L L L L L
L L L L L L L L L L L
L L L L L L L L L L L
L L L L L L L L L L L
L L L L L L L L L L L
L L L L L L L L L L L
L L L L L L L L L L L
'''

In [ ]:
'''
L L L L L L L L L L L
L L L L L L L L L L L
L L L L L L L L L L L
L L L L L L L L L L L
L L L L L L L L L L L
L L L L L L L L L L L
L L L L L L L L L L L
L L L L L L L L L L L
L 1 X L L L L L L L L
L 1 < 2 2 L L L L L L
L V F L L L L L L L L
'''

In [7]:
snakeobject11 = {
  "id": "sn1",
  "name": "sn1",
  "health": 100,
  "body": [{"x": 1, "y": 0}, {"x": 1, "y": 1}, {"x": 1, "y": 2}],
  "latency": "123",
  "head": {"x": 1, "y": 0},
  "length": 3,
  "shout": "s!",
  "squad": "1",
  "customizations":{"color":"#26CF04","head":"smile","tail":"bolt"}}

In [8]:
snakeobject21 = {
  "id": "sn2",
  "name": "sn2",
  "health": 100,
  "body": [{"x": 2, "y": 1}, {"x": 3, "y": 1}, {"x": 4, "y": 1}],
  "latency": "123",
  "head": {"x": 2, "y": 1},
  "length": 3,
  "shout": "s!",
  "squad": "2",
  "customizations":{"color":"#26CF04","head":"smile","tail":"bolt"}}

In [9]:
snakeobject12 = {
  "id": "sn1",
  "name": "sn1",
  "health": 100,
  "body": [{"x": 5, "y": 5}, {"x": 5, "y": 4}, {"x": 5, "y": 3}],
  "latency": "123",
  "head": {"x": 5, "y": 5},
  "length": 3,
  "shout": "s!",
  "squad": "1",
  "customizations":{"color":"#26CF04","head":"smile","tail":"bolt"}}

In [10]:
snakeobject22 = {
  "id": "sn2",
  "name": "sn2",
  "health": 100,
  "body": [{"x": 7, "y": 6}, {"x": 7, "y": 7}, {"x": 7, "y": 8}],
  "latency": "123",
  "head": {"x": 7, "y": 6},
  "length": 3,
  "shout": "s!",
  "squad": "2",
  "customizations":{"color":"#26CF04","head":"smile","tail":"bolt"}}

In [11]:
board1 = {"height": 11, "width": 11,
  "food": [{"x": 2, "y": 0}],
  "hazards": [{"x": 2, "y": 2}],
  "snakes": [snakeobject11, snakeobject21]}

In [12]:
board2 = {"height": 11, "width": 11,
  "food": [{}],
  "hazards": [{}],
  "snakes": [snakeobject12, snakeobject22]}

In [13]:
board3 = {"height": 11, "width": 11,
  "food": [{"x": 2, "y": 0}],
  "hazards": [{"x": 2, "y": 2}],
  "snakes": [snakeobject11, snakeobject21, snakeobject12, snakeobject22]}

In [160]:
def next_lob(board, lolos, counter):
    #board: board object
    #lolos: running list of snakes  #listof (listof snakes)
    snake = board['snakes'][counter:counter+1][0]
    
    #pull from board objects
    snakes = board["snakes"]        # listof [sn, sn, ...]
    hazards = board["hazards"]      # listof [({'x':_,'y':_}),...]
    board_height = board["height"]  # Natural
    board_width = board["width"]    # Natural
    foods = board["food"]           # listof [({'x':_,'y':_}),...]

    
    #possible_snakes is listof (listof {x,y})
    lolop_all4 = [
        [{"x": snake['head']['x'] , "y": snake['head']['y']+1}] + snake['body'][:],
        [{"x": snake['head']['x'] , "y": snake['head']['y']-1}] + snake['body'][:],
        [{"x": snake['head']['x']-1 , "y": snake['head']['y']}] + snake['body'][:],
        [{"x": snake['head']['x']+1 , "y": snake['head']['y']}] + snake['body'][:]
    ]
    
    #hits hazard or out of bounds
    lolop = []
    #print('lolop before: {}'.format(lolop))
    #print('lolop_all4 before: {}'.format(lolop_all4))

    for p in lolop_all4:
        #print('testing {}: {}'.format(snake['name'],p[0]))
        if (not ((p[0] in hazards) or
            (p[0]['x'] < 0) or
            (p[0]['x'] > (board_width - 1)) or 
            (p[0]['y'] < 0) or
            (p[0]['y'] > (board_height - 1)) or
            (p[0] in p[1:]))):

            lolop.append(p)

    #print('lolop after: {}'.format(lolop))

    #if not eating, move one space
    list_pos_food_adj = []
    foods_not_eaten = foods    
    for p in lolop:
        if p[0] in foods:
            list_pos_food_adj.append(p)
            #print(p)
        else:
            list_pos_food_adj.append(p[:-1])
            #print('pos is {}'.format(p[:-1]))
        
    for f in foods:
        for p in lolop:
            if f in p:
                foods_not_eaten.remove(f)

    #turn positions to snakes
    list_pos_to_snake = []
    for pos in list_pos_food_adj:
        list_pos_to_snake.append(
            [{
                "id": snake['id'],
                "name": snake['name'],
                "health": 54,
                "body": pos,
                "latency": snake['latency'],
                "head": pos[0],
                "length": len(pos),
                "shout": snake['shout'],
                "squad": snake['squad'],
                "customizations": snake['customizations']
            }]
        )

    next_lolos = [] #[[sn, sn, ...], [sn, sn, ...], [sn, sn, ...]]
    list_passed_snakes_bodies = []

    if len(lolos) == 0:
        next_lolos = list_pos_to_snake
    else:    
        for los in lolos: #starting list
            list_passed_snakes_bodies = []
            for s in los:
                list_passed_snakes_bodies.append(s['body'])
                
            for s in list_pos_to_snake: #current snake
                if (s[0]['head'] in list_passed_snakes_bodies[0]): #skip if collided
                    print('removed collisions')
                else:
                    next_lolos.append(los + s)
            
            print(list_passed_snakes_bodies)


    counter += 1
    if counter == len(snakes):
        lob = []
        for los in next_lolos:
            lob.append(
                {"height": board_height,
                "width": board_width,
                "food": foods_not_eaten,
                "hazards": hazards,
                "snakes": los 
                }
            )
        return lob
    return next_lob(board, next_lolos, counter)

In [166]:
board = board3
nl = next_lob(board,[],0)
len(nl)

removed collisions
[[{'x': 0, 'y': 0}, {'x': 1, 'y': 0}, {'x': 1, 'y': 1}]]
removed collisions
removed collisions
[[{'x': 2, 'y': 0}, {'x': 1, 'y': 0}, {'x': 1, 'y': 1}]]
[[{'x': 0, 'y': 0}, {'x': 1, 'y': 0}, {'x': 1, 'y': 1}], [{'x': 2, 'y': 0}, {'x': 2, 'y': 1}, {'x': 3, 'y': 1}]]
[[{'x': 0, 'y': 0}, {'x': 1, 'y': 0}, {'x': 1, 'y': 1}], [{'x': 2, 'y': 0}, {'x': 2, 'y': 1}, {'x': 3, 'y': 1}], [{'x': 5, 'y': 6}, {'x': 5, 'y': 5}, {'x': 5, 'y': 4}]]
[[{'x': 0, 'y': 0}, {'x': 1, 'y': 0}, {'x': 1, 'y': 1}], [{'x': 2, 'y': 0}, {'x': 2, 'y': 1}, {'x': 3, 'y': 1}], [{'x': 4, 'y': 5}, {'x': 5, 'y': 5}, {'x': 5, 'y': 4}]]
[[{'x': 0, 'y': 0}, {'x': 1, 'y': 0}, {'x': 1, 'y': 1}], [{'x': 2, 'y': 0}, {'x': 2, 'y': 1}, {'x': 3, 'y': 1}], [{'x': 6, 'y': 5}, {'x': 5, 'y': 5}, {'x': 5, 'y': 4}]]


9

In [168]:
nl

[{'height': 11,
  'width': 11,
  'food': [],
  'hazards': [{'x': 2, 'y': 2}],
  'snakes': [{'id': 'sn1',
    'name': 'sn1',
    'health': 54,
    'body': [{'x': 0, 'y': 0}, {'x': 1, 'y': 0}, {'x': 1, 'y': 1}],
    'latency': '123',
    'head': {'x': 0, 'y': 0},
    'length': 3,
    'shout': 's!',
    'squad': '1',
    'customizations': {'color': '#26CF04', 'head': 'smile', 'tail': 'bolt'}},
   {'id': 'sn2',
    'name': 'sn2',
    'health': 54,
    'body': [{'x': 2, 'y': 0}, {'x': 2, 'y': 1}, {'x': 3, 'y': 1}],
    'latency': '123',
    'head': {'x': 2, 'y': 0},
    'length': 3,
    'shout': 's!',
    'squad': '2',
    'customizations': {'color': '#26CF04', 'head': 'smile', 'tail': 'bolt'}},
   {'id': 'sn1',
    'name': 'sn1',
    'health': 54,
    'body': [{'x': 5, 'y': 6}, {'x': 5, 'y': 5}, {'x': 5, 'y': 4}],
    'latency': '123',
    'head': {'x': 5, 'y': 6},
    'length': 3,
    'shout': 's!',
    'squad': '1',
    'customizations': {'color': '#26CF04', 'head': 'smile', 'tail': 'bol